# Treinando seu modelo de vôlei

![](https://camo.githubusercontent.com/7eba6ff826871a5f1b9fb48d5dc7472dbf6bdbcda80bfb975bdde5a0ff71fdf5/68747470733a2f2f6f746f726f2e6e65742f696d672f736c696d6567796d2f706978656c2e676966)

In [ ]:
# Instala as bibliotecas necessárias
!pip install stable_baselines3
!pip install git+https://github.com/turing-usp/slimevolleygym

Vamos salvar os modelos treinados no seu drive: 😈😈😈

In [ ]:
from google.colab import drive

# Permite acesso ao seu drive
drive.mount('/content/drive')

In [ ]:
import os
import gym
import slimevolleygym
from slimevolleygym import SurvivalRewardEnv
from torch import nn

from stable_baselines3.ppo import PPO
from stable_baselines3.common.callbacks import EvalCallback

# Parâmetros para o treinamento
SEED = 721                            # Seed
NUM_TIMESTEPS = int(2e7)              # Número de timesteps de treino
EVAL_FREQ = 100000                    # A cada quantos timesteps o modelo é avaliado (ver EvalCallback)
EVAL_EPISODES = 100                   # Por quantos episódios o modelo é avaliado (ver EvalCallback)
LOGDIR = "/content/drive/MyDrive/ppo" # Local de salvar o modelo (no seu drive)

# Cria o ambiente
env = gym.make("SlimeVolley-v0")
env.seed(SEED)

# Arquitetura do modelo de PPO (usar caso somente tenha interesse em alterar o padrão de 64x64 neurônios)
model_arch=dict(
        log_std_init=-2,
        ortho_init=False,
        activation_fn=nn.ReLU,
        net_arch=[dict(pi=[64, 64], vf=[64, 64])] # possível trocar para [128, 128] e outros do tipo
      )

# Cria o modelo (mudar hiperparâmetros a vontade)
model = PPO('MlpPolicy', env, n_steps=4096, batch_size=32, ent_coef=0.005, n_epochs=10,
            learning_rate=3e-4, clip_range=0.2, gamma=0.99, gae_lambda=0.95, verbose=2
            # , policy_kwargs = model_arch # Descomentar caso tenha interesse em usar outras arquiteturas!!!
            )

# Carrega modelo salvo caso já exista
if os.path.exists(LOGDIR + "/best_model.zip"):
  model = PPO.load(LOGDIR + "/best_model", env=env, n_steps=4096, batch_size=32, ent_coef=0.005, n_epochs=10,
            learning_rate=3e-4, clip_range=0.2, gamma=0.99, gae_lambda=0.95, verbose=2
            # , policy_kwargs = model_arch # Descomentar caso tenha interesse em usar outras arquiteturas!!!
            )

# Salva o melhor modelo a cada avaliação
eval_callback = EvalCallback(env, best_model_save_path=LOGDIR, log_path=LOGDIR, eval_freq=EVAL_FREQ, n_eval_episodes=EVAL_EPISODES)

# Treinamento do Modelo
model.learn(total_timesteps=NUM_TIMESTEPS, callback=eval_callback)

model.save(os.path.join(LOGDIR, "final_model"))

env.close()